In [29]:
import os

### Model Component

In [3]:
from openai import OpenAI

In [4]:
client = OpenAI()

client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {
            "role": "user",
            "content": "2002년 월드컵 4강 국가를 알려줘."
        }
    ]
)

ChatCompletion(id='chatcmpl-ATIO7Abka0o4FCO1Io1yBoe8hgEoY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2002년 월드컵 4강에는 프랑스, 브라질, 독일, 대한민국이 참가했습니다. 결승에서 브라질이 독일을 이기고 우승을 차지했습니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731545559, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=76, prompt_tokens=33, total_tokens=109, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [5]:
from langchain_openai import ChatOpenAI

In [7]:
chat = ChatOpenAI(
    model='gpt-4o-mini'
)

chat.invoke("안녕! 너를 소개해줄래?")

AIMessage(content='안녕하세요! 저는 인공지능 언어 모델인 ChatGPT입니다. 다양한 주제에 대해 질문에 답하고, 정보 제공, 대화, 글쓰기 도움 등을 할 수 있습니다. 언어와 관련된 다양한 작업을 지원하며, 여러분이 필요로 하는 정보를 최대한 잘 전달하려 노력하고 있습니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 17, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd3165e4-a322-47ca-87ff-1d36b6acaeab-0', usage_metadata={'input_tokens': 17, 'output_tokens': 73, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### PromptTemplate

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
prompt = (
    PromptTemplate.from_template(
        """
        너는 훌륭한 요리사야. 내가 가진 재료들을 활용해서 만들 수 있는 요리를 {개수}추천하고,
        그 요리의 레시피를 알려줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
        )
    )

prompt

PromptTemplate(input_variables=['개수', '재료'], input_types={}, partial_variables={}, template='\n        너는 훌륭한 요리사야. 내가 가진 재료들을 활용해서 만들 수 있는 요리를 {개수}추천하고,\n        그 요리의 레시피를 알려줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        {재료}\n        ')

In [11]:
prompt.invoke({"개수": 4, "재료": "사과, 딸기, 잼"})

StringPromptValue(text='\n        너는 훌륭한 요리사야. 내가 가진 재료들을 활용해서 만들 수 있는 요리를 4추천하고,\n        그 요리의 레시피를 알려줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        사과, 딸기, 잼\n        ')

In [12]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # SystemMessage
        ("system", "You are a helpful AI bot. Your name is {name}."),
        # HumanMessage, AIMessage
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        # HumanMessage
        ("human", "{user_input}"),
    ]
)

In [15]:
messages = chat_template.format_messages(name='Bob', user_input='What is your name?')
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


### LCEL

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [17]:
# Prompt template
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# LLM
model = ChatOpenAI(model='gpt-4o-mini')

# LCEL
chain = prompt | model | StrOutputParser()

# invoke
chain.invoke({'topic': 'ice cream'})

'Why did the ice cream truck break down?  \n\nBecause it had a rocky road!'

In [18]:
# Define chain
model = ChatOpenAI(model='gpt-4o')
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | model

# streaming
for s in chain.stream({'topic': 'bears'}):
    print(s.content, end="", flush=True) # flush=True : 버퍼에 잠시 저장하지 않고 즉시 출력

Why don’t bears wear shoes?  

Because they prefer bear feet!

### OutputParser

In [19]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

In [28]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)

chain = chat_template | llm
chain.invoke("코미디")

AIMessage(content='["슈렉", "행오버", "미국의 아이들"]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 73, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-ee96b1fb-56a9-4c5c-b22b-b3bc6f57dfd6-0', usage_metadata={'input_tokens': 73, 'output_tokens': 16, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [31]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

In [32]:
# Define CSV parser
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [35]:
prompt = PromptTemplate(
    template="List {subject}. Answer in korean \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

model = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | model | output_parser
chain.invoke({"subject": "호러 영화"})

['컨저링',
 '악마의 소원',
 '사탄의 인형',
 '할로윈',
 '미드소마',
 '더 바이더스',
 '그 아래',
 '링',
 '귀향',
 '고스트 스토리']

In [38]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [39]:
class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라 인구(int 형식)")
    social: str = Field(description="사용자가 물어본 나라의 최근 이슈, 현재 날짜 2024년 11월 14일")

In [43]:
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query. \n{format_instructions} \n{query} \n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | model | parser

country_query = "아르헨티나에 대해 한글로 알려줘."
chain.invoke({"query": country_query})

{'continent': '남아메리카',
 'population': '46000000',
 'social': '2024년 현재 아르헨티나는 경제 위기와 인플레이션 문제로 어려움을 겪고 있으며, 정치적 불안정성 또한 큰 이슈입니다.'}

### Runnable of LCEL

#### RunnablePassthrough

In [44]:
from langchain_core.runnables import RunnablePassthrough

In [45]:
RunnablePassthrough().invoke("안녕하세요.")

'안녕하세요.'

In [46]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [47]:
prompt = ChatPromptTemplate.from_template("""다음 주어지는 한글 문장을 독일어로 번역해줘 {sentence}
German sentence: (print from here)""")

model = ChatOpenAI(model='gpt-4o-mini')

output_parser = StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | output_parser
runnable_chain.invoke({"sentence": "그는 매일 아침 일찍 일어납니다."})

'Er steht jeden Morgen früh auf.'

In [48]:
(RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)).invoke({"num": 3})

{'num': 3, 'mult': 9}

In [49]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [50]:
runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

#### RunnableLambda

In [51]:
def add_smile(x):
    return x + ':)'

In [52]:
from langchain_core.runnables import RunnableLambda

add_smile = RunnableLambda(add_smile)

In [53]:
add_smile

RunnableLambda(add_smile)

In [55]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
# from langchain.schema.output_parser import StrOutputParser

In [56]:
prompt_str = "{topic}의 역사에 대해 간략하게 설명해주세요."
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model='gpt-4o')

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [57]:
from langchain_core.runnables import RunnableLambda

In [58]:
def add_thanks(x):
    return x + " 들어주셔서 감사합니다! :)"

In [59]:
add_thanks = RunnableLambda(add_thanks)

In [60]:
chain = prompt | model | output_parser | add_thanks
chain.invoke('반도체')

'반도체의 역사는 전자공학과 정보기술의 발전과 깊은 연관이 있습니다. 다음은 반도체의 주요 역사적 발전 과정을 간략하게 정리한 것입니다.\n\n1. **초기 발견 및 개념(19세기 후반)**: 반도체의 개념은 19세기 후반에 처음으로 등장했습니다. 1833년에는 마이클 패러데이가 실험을 통해 황화은의 저항이 온도에 따라 변하는 것을 발견했습니다.\n\n2. **진공관 시대(1900년대 초반)**: 반도체가 본격적으로 주목받기 전, 전자기기는 주로 진공관에 의존했습니다. 진공관은 전류를 증폭하고 스위칭하는 데 사용되었지만, 크기가 크고 효율이 낮았습니다.\n\n3. **트랜지스터의 발명(1947년)**: 1947년 벨 연구소의 존 바딘, 월터 브래튼, 윌리엄 쇼클리가 트랜지스터를 발명하면서 반도체 기술은 획기적인 전환점을 맞았습니다. 트랜지스터는 진공관을 대체할 수 있는 소형, 저전력, 고효율 소자로, 전자기기의 소형화와 효율성 향상에 크게 기여했습니다.\n\n4. **집적회로(IC)의 개발(1958년)**: 1958년 잭 킬비와 로버트 노이스가 각각 독립적으로 집적회로를 개발했습니다. 이는 여러 개의 트랜지스터와 기타 전자 부품을 하나의 칩에 통합할 수 있게 하여, 전자 기기의 성능을 크게 향상시키고 제조 비용을 절감시켰습니다.\n\n5. **마이크로프로세서의 탄생(1970년대)**: 1971년 인텔이 최초의 상용 마이크로프로세서인 4004를 발표하면서 컴퓨터의 발전에 큰 기여를 했습니다. 이는 컴퓨터 및 디지털 기기의 소형화와 대중화에 중요한 역할을 했습니다.\n\n6. **현대 반도체 기술(1990년대 이후)**: 1990년대부터 현재까지 반도체 기술은 나노미터 공정 기술을 통해 지속적으로 발전하고 있습니다. 이로 인해 컴퓨터, 스마트폰, IoT 기기 등 다양한 분야에서 고성능, 저전력 반도체가 활용되고 있습니다.\n\n반도체 기술의 발전은 현대 정보화 사회를 가능하게 한 핵심 요소 중 하나로, 앞으로도 기술 혁신과 산업 발전에 중요한 역할을 할 것으로 기

In [61]:
from IPython.display import display, Markdown

display(Markdown('반도체의 역사는 전자공학과 정보기술의 발전과 깊은 연관이 있습니다. 다음은 반도체의 주요 역사적 발전 과정을 간략하게 정리한 것입니다.\n\n1. **초기 발견 및 개념(19세기 후반)**: 반도체의 개념은 19세기 후반에 처음으로 등장했습니다. 1833년에는 마이클 패러데이가 실험을 통해 황화은의 저항이 온도에 따라 변하는 것을 발견했습니다.\n\n2. **진공관 시대(1900년대 초반)**: 반도체가 본격적으로 주목받기 전, 전자기기는 주로 진공관에 의존했습니다. 진공관은 전류를 증폭하고 스위칭하는 데 사용되었지만, 크기가 크고 효율이 낮았습니다.\n\n3. **트랜지스터의 발명(1947년)**: 1947년 벨 연구소의 존 바딘, 월터 브래튼, 윌리엄 쇼클리가 트랜지스터를 발명하면서 반도체 기술은 획기적인 전환점을 맞았습니다. 트랜지스터는 진공관을 대체할 수 있는 소형, 저전력, 고효율 소자로, 전자기기의 소형화와 효율성 향상에 크게 기여했습니다.\n\n4. **집적회로(IC)의 개발(1958년)**: 1958년 잭 킬비와 로버트 노이스가 각각 독립적으로 집적회로를 개발했습니다. 이는 여러 개의 트랜지스터와 기타 전자 부품을 하나의 칩에 통합할 수 있게 하여, 전자 기기의 성능을 크게 향상시키고 제조 비용을 절감시켰습니다.\n\n5. **마이크로프로세서의 탄생(1970년대)**: 1971년 인텔이 최초의 상용 마이크로프로세서인 4004를 발표하면서 컴퓨터의 발전에 큰 기여를 했습니다. 이는 컴퓨터 및 디지털 기기의 소형화와 대중화에 중요한 역할을 했습니다.\n\n6. **현대 반도체 기술(1990년대 이후)**: 1990년대부터 현재까지 반도체 기술은 나노미터 공정 기술을 통해 지속적으로 발전하고 있습니다. 이로 인해 컴퓨터, 스마트폰, IoT 기기 등 다양한 분야에서 고성능, 저전력 반도체가 활용되고 있습니다.\n\n반도체 기술의 발전은 현대 정보화 사회를 가능하게 한 핵심 요소 중 하나로, 앞으로도 기술 혁신과 산업 발전에 중요한 역할을 할 것으로 기대됩니다. 들어주셔서 감사합니다! :)'))

반도체의 역사는 전자공학과 정보기술의 발전과 깊은 연관이 있습니다. 다음은 반도체의 주요 역사적 발전 과정을 간략하게 정리한 것입니다.

1. **초기 발견 및 개념(19세기 후반)**: 반도체의 개념은 19세기 후반에 처음으로 등장했습니다. 1833년에는 마이클 패러데이가 실험을 통해 황화은의 저항이 온도에 따라 변하는 것을 발견했습니다.

2. **진공관 시대(1900년대 초반)**: 반도체가 본격적으로 주목받기 전, 전자기기는 주로 진공관에 의존했습니다. 진공관은 전류를 증폭하고 스위칭하는 데 사용되었지만, 크기가 크고 효율이 낮았습니다.

3. **트랜지스터의 발명(1947년)**: 1947년 벨 연구소의 존 바딘, 월터 브래튼, 윌리엄 쇼클리가 트랜지스터를 발명하면서 반도체 기술은 획기적인 전환점을 맞았습니다. 트랜지스터는 진공관을 대체할 수 있는 소형, 저전력, 고효율 소자로, 전자기기의 소형화와 효율성 향상에 크게 기여했습니다.

4. **집적회로(IC)의 개발(1958년)**: 1958년 잭 킬비와 로버트 노이스가 각각 독립적으로 집적회로를 개발했습니다. 이는 여러 개의 트랜지스터와 기타 전자 부품을 하나의 칩에 통합할 수 있게 하여, 전자 기기의 성능을 크게 향상시키고 제조 비용을 절감시켰습니다.

5. **마이크로프로세서의 탄생(1970년대)**: 1971년 인텔이 최초의 상용 마이크로프로세서인 4004를 발표하면서 컴퓨터의 발전에 큰 기여를 했습니다. 이는 컴퓨터 및 디지털 기기의 소형화와 대중화에 중요한 역할을 했습니다.

6. **현대 반도체 기술(1990년대 이후)**: 1990년대부터 현재까지 반도체 기술은 나노미터 공정 기술을 통해 지속적으로 발전하고 있습니다. 이로 인해 컴퓨터, 스마트폰, IoT 기기 등 다양한 분야에서 고성능, 저전력 반도체가 활용되고 있습니다.

반도체 기술의 발전은 현대 정보화 사회를 가능하게 한 핵심 요소 중 하나로, 앞으로도 기술 혁신과 산업 발전에 중요한 역할을 할 것으로 기대됩니다. 들어주셔서 감사합니다! :)

#### RunnableParallel

In [62]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [63]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

In [64]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_thanks
)

runnable.invoke("안녕하세요.")

{'passed': '안녕하세요.', 'modified': '안녕하세요. 들어주셔서 감사합니다! :)'}

In [65]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [67]:
model = ChatOpenAI(model='gpt-4o', max_tokens=128, temperature=0)

histoy_prompt = ChatPromptTemplate.from_template("{topic}가 무엇의 약자인지 알려주세요.")
celeb_prompt = ChatPromptTemplate.from_template("{topic} 분야의 유명인사 3명의 이름을 알려주세요.")

output_parser = StrOutputParser()

history_chain = histoy_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

map_chain = RunnableParallel(
    history=history_chain,
    celeb=celeb_chain
)

result = map_chain.invoke({"topic": "AI"})
result

{'history': 'AI는 "Artificial Intelligence"의 약자입니다. 이는 한국어로 "인공지능"을 의미하며, 기계나 컴퓨터 시스템이 인간과 유사한 지능적 행동을 수행할 수 있도록 하는 기술을 말합니다.',
 'celeb': 'AI 분야에서 유명한 인사로는 다음과 같은 사람들이 있습니다:\n\n1. **Geoffrey Hinton**: 인공신경망과 딥러닝 분야의 선구자로, "딥러닝의 아버지"로 불리기도 합니다. 그의 연구는 현대 AI 발전에 큰 기여를 했습니다.\n\n2. **Yann LeCun**: 딥러닝과 컴퓨터 비전 분야의 선구자 중 한 명으로, 특히 합성곱 신경망(CNN)의 개발로 유명합니다. 현재 Meta(구 Facebook)의 AI 연구 책임자로 활동하고'}